# Question 1 (15 Marks)
Build a RNN based seq2seq model which contains the following layers: (i) input layer for character embeddings (ii) one encoder RNN which sequentially encodes the input character sequence (Latin) (iii) one decoder RNN which takes the last state of the encoder as input and produces one output character at a time (Devanagari). 

The code should be flexible such that the dimension of the input character embeddings, the hidden states of the encoders and decoders, the cell (RNN, LSTM, GRU) and the number of layers in the encoder and decoder can be changed.

# Question 2 (10 Marks)

You will now train your model using any one language from the [Dakshina dataset](https://github.com/google-research-datasets/dakshina) (I would suggest pick a language that you can read so that it is easy to analyse the errors). Use the standard train, dev, test set from the folder dakshina_dataset_v1.0/hi/lexicons/ (replace hi by the language of your choice)

Using the sweep feature in wandb find the best hyperparameter configuration. Here are some suggestions but you are free to decide which hyperparameters you want to explore


- input embedding size: 16, 32, 64, 256, ...



- number of encoder layers: 1, 2, 3 



- number of decoder layers: 1, 2, 3 



- hidden layer size: 16, 32, 64, 256, ...



- cell type: RNN, GRU, LSTM



- dropout: 20%, 30% (btw, where will you add dropout? you should read up a bit on this)



- beam search in decoder with different beam sizes: 


Based on your sweep please paste the following plots which are automatically generated by wandb:

- accuracy v/s created plot (I would like to see the number of experiments you ran to get the best configuration). 

- parallel co-ordinates plot

- correlation summary table (to see the correlation of each hyperparameter with the loss/accuracy)

Also write down the hyperparameters and their values that you sweeped over. Smart strategies to reduce the number of runs while still achieving a high accuracy would be appreciated. Write down any unique strategy that you tried for efficiently searching the hyperparameters.

In [ ]:
import wandb
import torch
import torch.nn as nn
import torch.optim as optim
import random
from torch.nn.utils.rnn import pad_sequence

In [ ]:
# Log in to W&B (Weights and Biases) for experiment tracking
wandb.login(key='acdc26d2fc17a56e83ea3ae6c10e496128dee648')


# ---------- Model Definitions ----------

# Encoder class for the Seq2Seq model
class Encoder(nn.Module):
    def __init__(self, input_dim, embed_dim, hidden_dim, num_layers, cell_type='LSTM', dropout=0.2, bidirectional=False):
        super(Encoder, self).__init__()
        # Embedding layer for input tokens with padding_idx to ignore padding during training
        self.embedding = nn.Embedding(input_dim, embed_dim, padding_idx=0)

        # Select the RNN cell type: RNN, LSTM, or GRU
        rnn_cls = {'RNN': nn.RNN, 'LSTM': nn.LSTM, 'GRU': nn.GRU}[cell_type]

        # Define the RNN layer with given configuration
        self.rnn = rnn_cls(embed_dim, hidden_dim, num_layers, dropout=dropout, batch_first=True, bidirectional=bidirectional)
        self.cell_type = cell_type
        self.bidirectional = bidirectional

    def forward(self, src):
        # Apply embedding on source sequence
        embedded = self.embedding(src)

        # Pass through the RNN and return only the hidden state(s)
        outputs, hidden = self.rnn(embedded)
        return hidden


# Decoder class for the Seq2Seq model
class Decoder(nn.Module):
    def __init__(self, output_dim, embed_dim, hidden_dim, num_layers, cell_type='LSTM', dropout=0.2, bidirectional=False):
        super(Decoder, self).__init__()
        # Embedding layer for output tokens
        self.embedding = nn.Embedding(output_dim, embed_dim, padding_idx=0)

        # Select the RNN cell type
        rnn_cls = {'RNN': nn.RNN, 'LSTM': nn.LSTM, 'GRU': nn.GRU}[cell_type]

        # RNN for decoding
        self.rnn = rnn_cls(embed_dim, hidden_dim, num_layers, dropout=dropout, batch_first=True, bidirectional=bidirectional)

        # Final fully connected layer to project hidden states to vocabulary size
        self.fc_out = nn.Linear(hidden_dim * (2 if bidirectional else 1), output_dim)
        self.cell_type = cell_type
        self.bidirectional = bidirectional

    def forward(self, input, hidden):
        # Add time dimension (B, 1) since we're decoding one step at a time
        input = input.unsqueeze(1)

        # Embed the input token
        embedded = self.embedding(input)

        # Pass through the RNN
        output, hidden = self.rnn(embedded, hidden)

        # Remove time dimension and pass through linear layer
        output = self.fc_out(output.squeeze(1))
        return output, hidden


# Seq2Seq model that combines Encoder and Decoder
class Seq2Seq(nn.Module):
    def __init__(self, input_dim, output_dim, embed_dim, hidden_dim, enc_layers, dec_layers,
                 cell_type='LSTM', dropout=0.2, bidirectional=False):
        super(Seq2Seq, self).__init__()
        # Initialize encoder and decoder
        self.encoder = Encoder(input_dim, embed_dim, hidden_dim, enc_layers, cell_type, dropout, bidirectional)
        self.decoder = Decoder(output_dim, embed_dim, hidden_dim, dec_layers, cell_type, dropout, bidirectional)
        self.cell_type = cell_type

    # Adjust hidden state dimensions if encoder and decoder have different number of layers
    def adjust_hidden_for_decoder(self, encoder_hidden):
        enc_layers = self.encoder.rnn.num_layers
        dec_layers = self.decoder.rnn.num_layers

        def adjust(h):
            if enc_layers == dec_layers:
                return h
            elif enc_layers < dec_layers:
                # Repeat last encoder hidden layer to match decoder layers
                repeat_h = h[-1].unsqueeze(0).repeat(dec_layers - enc_layers, 1, 1)
                return torch.cat([h, repeat_h], dim=0)
            else:
                # Truncate encoder hidden state to match decoder layers
                return h[-dec_layers:]

        # If using LSTM, adjust both hidden state (h) and cell state (c)
        if self.cell_type == 'LSTM':
            h, c = encoder_hidden
            h = adjust(h)
            c = adjust(c)
            return (h, c)
        else:
            h = encoder_hidden
            h = adjust(h)
            return h

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size, trg_len = trg.size()

        # Initialize tensor to store decoder outputs
        outputs = torch.zeros(batch_size, trg_len, self.decoder.fc_out.out_features, device=src.device)

        # Pass source through encoder
        hidden = self.encoder(src)

        # Adjust encoder hidden states to fit decoder
        decoder_hidden = self.adjust_hidden_for_decoder(hidden)

        # First input to the decoder is the <sos> token
        input = trg[:, 0]

        # Decode each time step
        for t in range(1, trg_len):
            output, decoder_hidden = self.decoder(input, decoder_hidden)
            outputs[:, t] = output

            # Decide whether to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1)  # Get most probable token

            # Next input is ground truth if teacher forcing, else prediction
            input = trg[:, t] if teacher_force else top1

        return outputs



# ---------- Utility Functions ----------

# Build character vocabulary from dataset
def build_vocab(sequences):
    chars = set(ch for seq in sequences for ch in seq)  # Unique characters in dataset
    stoi = {'<pad>': 0, '<sos>': 1, '<eos>': 2, '<unk>': 3}  # Special tokens
    for ch in sorted(chars):
        stoi[ch] = len(stoi)
    itos = {i: ch for ch, i in stoi.items()}
    return stoi, itos

# Encode a sequence using string-to-index mapping
def encode_sequence(seq, stoi):
    return [stoi.get(c, stoi['<unk>']) for c in seq]  # Replace unknown chars with <unk>

# Prepare a batch of input-output sequences for training
def prepare_batch(pairs, inp_stoi, out_stoi, device):
    # Convert each sequence into tensor and add special tokens
    src_seq = [torch.tensor(encode_sequence(src, inp_stoi) + [inp_stoi['<eos>']]) for src, _ in pairs]
    trg_seq = [torch.tensor([out_stoi['<sos>']] + encode_sequence(trg, out_stoi) + [out_stoi['<eos>']]) for _, trg in pairs]

    # Pad sequences to make them the same length
    src_batch = pad_sequence(src_seq, batch_first=True, padding_value=inp_stoi['<pad>'])
    trg_batch = pad_sequence(trg_seq, batch_first=True, padding_value=out_stoi['<pad>'])

    return src_batch.to(device), trg_batch.to(device)

# Read a tab-separated file and return pairs (input, output)
def read_dataset(path):
    with open(path, encoding='utf-8') as f:
        lines = f.read().strip().split('\n')
        return [(l.split('\t')[1], l.split('\t')[0]) for l in lines if '\t' in l]

# Calculate word-level accuracy (entire sequences must match, excluding padding)
def calculate_word_accuracy(preds, targets, ignore_index=0):
    # Get predicted token indices (highest probability)
    preds = preds.argmax(dim=-1)  # Shape: [batch, seq_len]
    
    # Mask to ignore padding tokens
    mask = targets != ignore_index

    # Apply mask to predictions and targets
    preds_masked = preds * mask
    targets_masked = targets * mask

    # Compare entire sequences for exact match
    sequence_correct = (preds_masked == targets_masked).all(dim=1)
    
    # Compute accuracy as percentage of sequences that are fully correct
    word_accuracy = sequence_correct.float().mean().item() * 100

    return word_accuracy

# Evaluate the model on a dataset
def evaluate(model, data, src_vocab, tgt_vocab, device, criterion, batch_size):
    model.eval()
    total_loss = 0
    total_acc = 0

    with torch.no_grad():
        for i in range(0, len(data), batch_size):
            batch = data[i:i + batch_size]
            src, trg = prepare_batch(batch, src_vocab, tgt_vocab, device)

            # Forward pass
            output = model(src, trg)

            # Compute loss ignoring the <sos> token (at position 0)
            loss = criterion(output[:, 1:].reshape(-1, output.shape[-1]), trg[:, 1:].reshape(-1))

            # Compute word-level accuracy
            acc = calculate_word_accuracy(output[:, 1:], trg[:, 1:])

            total_loss += loss.item()
            total_acc += acc

    # Return average loss and accuracy over all batches
    return total_loss / len(data), total_acc / (len(data) // batch_size)


# Train on train dataset

In [3]:
# ---------- Train Function ----------

def train():
    # Initialize a Weights & Biases (wandb) run with configuration for hyperparameters
    wandb.init(config={
        "embed_dim": 128,                  # Size of embedding vectors
        "hidden_dim": 256,                 # Hidden layer size for encoder/decoder
        "enc_layers": 2,                   # Number of encoder layers
        "dec_layers": 2,                   # Number of decoder layers
        "cell_type": "LSTM",               # Type of RNN cell (LSTM, GRU, RNN)
        "dropout": 0.2,                    # Dropout rate
        "epochs": 10,                      # Number of training epochs
        "batch_size": 64,                  # Batch size
        "bidirectional": False,            # Whether to use bidirectional encoder
        "learning_rate": 0.001,            # Learning rate for optimizer
        "optimizer": "adam",               # Optimizer to use (adam or nadam)
        "teacher_forcing_ratio": 0.5,      # Probability of using teacher forcing
        "beam_width": 1                    # Beam width for decoding (not used during training)
    })

    # Generate a unique name for the current run based on the config
    run_name = (
        f"{wandb.config.cell_type}_embed{wandb.config.embed_dim}_"
        f"hid{wandb.config.hidden_dim}_enc{wandb.config.enc_layers}_"
        f"dec{wandb.config.dec_layers}_drop{wandb.config.dropout}_"
        f"bs{wandb.config.batch_size}_lr{wandb.config.learning_rate}_"
        f"opt-{wandb.config.optimizer}_tf{wandb.config.teacher_forcing_ratio}_"
        f"bi-{wandb.config.bidirectional}_bw{wandb.config.beam_width}"
    )
    wandb.run.name = run_name

    # Get configuration and set device to GPU if available
    config = wandb.config
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Load training and validation data
    train_data = read_dataset("/kaggle/input/dakshina-dataset/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.train.tsv")
    dev_data = read_dataset("/kaggle/input/dakshina-dataset/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.dev.tsv")

    # Build vocabularies from training source and target data
    src_vocab, tgt_vocab = build_vocab([src for src, _ in train_data]), build_vocab([tgt for _, tgt in train_data])

    # Initialize Seq2Seq model with specified configuration
    model = Seq2Seq(len(src_vocab[0]), len(tgt_vocab[0]), config.embed_dim, config.hidden_dim,
                    config.enc_layers, config.dec_layers, config.cell_type, config.dropout, config.bidirectional).to(device)

    # Choose optimizer based on config
    if config.optimizer == "adam":
        optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
    elif config.optimizer == "nadam":
        optimizer = optim.NAdam(model.parameters(), lr=config.learning_rate)
    else:
        raise ValueError("Unsupported optimizer")

    # Use CrossEntropyLoss for training, ignoring the padding index (assumed to be 0)
    criterion = nn.CrossEntropyLoss(ignore_index=0)

    # Start training loop over epochs
    for epoch in range(config.epochs):
        model.train()                          # Set model to training mode
        total_loss = 0                         # Accumulate total loss for the epoch
        total_acc = 0                          # Accumulate total accuracy for the epoch
        random.shuffle(train_data)             # Shuffle training data at start of each epoch

        for i in range(0, len(train_data), config.batch_size):
            batch = train_data[i:i + config.batch_size]   # Get batch of data
            src, trg = prepare_batch(batch, src_vocab[0], tgt_vocab[0], device)  # Prepare tensors

            optimizer.zero_grad()             # Reset gradients
            output = model(src, trg, teacher_forcing_ratio=config.teacher_forcing_ratio)  # Forward pass

            # Compute loss (excluding first token)
            loss = criterion(output[:, 1:].reshape(-1, output.shape[-1]), trg[:, 1:].reshape(-1))
            
            # Compute accuracy of predicted words
            acc = calculate_word_accuracy(output[:, 1:], trg[:, 1:])

            loss.backward()                   # Backpropagation
            optimizer.step()                  # Optimizer update

            total_loss += loss.item()         # Accumulate loss
            total_acc += acc                  # Accumulate accuracy

        # Compute average loss and accuracy for training
        avg_train_loss = total_loss / len(train_data)
        avg_train_acc = total_acc / (len(train_data) // config.batch_size)

        # Evaluate model on validation set
        val_loss, val_acc = evaluate(model, dev_data, src_vocab[0], tgt_vocab[0], device, criterion, config.batch_size)

        # Log metrics to wandb
        wandb.log({
            "Train Loss": avg_train_loss,
            "Train Accuracy": avg_train_acc,
            "Validation Loss": val_loss,
            "Validation Accuracy": val_acc,
            "Epoch": epoch + 1,
            "Learning Rate": config.learning_rate,
            "Teacher Forcing Ratio": config.teacher_forcing_ratio,
            "Optimizer": config.optimizer,
            "Bidirectional": config.bidirectional,
            "Beam Width": config.beam_width
        })

        # Print epoch summary
        print(f"Epoch {epoch + 1}/{config.epochs} | Train Loss: {avg_train_loss:.4f}, Train Acc: {avg_train_acc:.2f}% | Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

    # Finish wandb run
    wandb.finish()

# ---------- Sweep Setup ----------

# Configuration for hyperparameter sweep using Bayesian optimization
sweep_config = {
    'method': 'bayes',   # Use Bayesian optimization to select next set of hyperparameters
    'metric': {'name': 'Validation Accuracy', 'goal': 'maximize'},  # Target metric

    # Define search space for hyperparameters
    'parameters': {
        'embed_dim': {'values': [32, 64, 256]},
        'hidden_dim': {'values': [64, 128]},
        'enc_layers': {'values': [1,2,3]},
        'dec_layers': {'values': [1,2,3]},
        'cell_type': {'values': ['LSTM','GRU','RNN']},
        'dropout': {'values': [0.2, 0.3]},
        'batch_size': {'values': [32,64]},
        'epochs': {'values': [5,10,15]},
        'bidirectional': {'values': [False]},
        'learning_rate': {'values': [0.001, 0.002, 0.0001]},
        'optimizer': {'values': ['adam', 'nadam']},
        'teacher_forcing_ratio': {'values': [0.2, 0.5, 0.7]},
        'beam_width': {'values': [1, 3, 5]}
    }
}

# Initialize a sweep with the defined configuration under the specified project name
sweep_id = wandb.sweep(sweep_config, project="without_attention_sweep")

# Launch sweep agent to run the `train` function multiple times with different hyperparameter combinations
wandb.agent(sweep_id, function=train, count=50)


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: k5x17vt3
Sweep URL: https://wandb.ai/viinod9-iitm/without_attention_sweep/sweeps/k5x17vt3


wandb: Agent Starting Run: ic3zb24c with config:
wandb: 	batch_size: 64
wandb: 	beam_width: 3
wandb: 	bidirectional: False
wandb: 	cell_type: LSTM
wandb: 	dec_layers: 1
wandb: 	dropout: 0.3
wandb: 	embed_dim: 32
wandb: 	enc_layers: 1
wandb: 	epochs: 10
wandb: 	hidden_dim: 64
wandb: 	learning_rate: 0.0001
wandb: 	optimizer: adam
wandb: 	teacher_forcing_ratio: 0.7


Epoch 1/10 | Train Loss: 0.0547, Train Acc: 0.00% | Val Loss: 0.0502, Val Acc: 0.00%
Epoch 2/10 | Train Loss: 0.0479, Train Acc: 0.00% | Val Loss: 0.0478, Val Acc: 0.00%
Epoch 3/10 | Train Loss: 0.0453, Train Acc: 0.00% | Val Loss: 0.0457, Val Acc: 0.00%
Epoch 4/10 | Train Loss: 0.0431, Train Acc: 0.01% | Val Loss: 0.0443, Val Acc: 0.00%
Epoch 5/10 | Train Loss: 0.0417, Train Acc: 0.02% | Val Loss: 0.0432, Val Acc: 0.00%
Epoch 6/10 | Train Loss: 0.0404, Train Acc: 0.03% | Val Loss: 0.0420, Val Acc: 0.05%
Epoch 7/10 | Train Loss: 0.0392, Train Acc: 0.07% | Val Loss: 0.0406, Val Acc: 0.07%
Epoch 8/10 | Train Loss: 0.0380, Train Acc: 0.12% | Val Loss: 0.0400, Val Acc: 0.07%
Epoch 9/10 | Train Loss: 0.0369, Train Acc: 0.18% | Val Loss: 0.0388, Val Acc: 0.07%
Epoch 10/10 | Train Loss: 0.0358, Train Acc: 0.24% | Val Loss: 0.0379, Val Acc: 0.14%


Beam Width,▁▁▁▁▁▁▁▁▁▁
Epoch,▁▂▃▃▄▅▆▆▇█
Learning Rate,▁▁▁▁▁▁▁▁▁▁
Teacher Forcing Ratio,▁▁▁▁▁▁▁▁▁▁
Train Accuracy,▁▁▁▁▂▂▃▄▆█
Train Loss,█▅▅▄▃▃▂▂▁▁
Validation Accuracy,▁▁▁▁▁▃▅▅▅█
Validation Loss,█▇▅▅▄▃▃▂▂▁
Beam Width,3
Bidirectional,False
Epoch,10


...

Epoch 1/15 | Train Loss: 0.0417, Train Acc: 0.25% | Val Loss: 0.0306, Val Acc: 1.61%
Epoch 2/15 | Train Loss: 0.0220, Train Acc: 8.33% | Val Loss: 0.0187, Val Acc: 15.90%
Epoch 3/15 | Train Loss: 0.0153, Train Acc: 18.41% | Val Loss: 0.0154, Val Acc: 24.07%
Epoch 4/15 | Train Loss: 0.0127, Train Acc: 24.73% | Val Loss: 0.0143, Val Acc: 27.45%
Epoch 5/15 | Train Loss: 0.0110, Train Acc: 29.37% | Val Loss: 0.0135, Val Acc: 29.66%
Epoch 6/15 | Train Loss: 0.0101, Train Acc: 32.83% | Val Loss: 0.0132, Val Acc: 31.38%
Epoch 7/15 | Train Loss: 0.0092, Train Acc: 35.91% | Val Loss: 0.0125, Val Acc: 33.95%
Epoch 8/15 | Train Loss: 0.0085, Train Acc: 38.82% | Val Loss: 0.0128, Val Acc: 33.10%
Epoch 9/15 | Train Loss: 0.0080, Train Acc: 41.26% | Val Loss: 0.0125, Val Acc: 34.05%
Epoch 10/15 | Train Loss: 0.0075, Train Acc: 43.74% | Val Loss: 0.0119, Val Acc: 34.44%
Epoch 11/15 | Train Loss: 0.0072, Train Acc: 45.48% | Val Loss: 0.0120, Val Acc: 36.41%
Epoch 12/15 | Train Loss: 0.0067, Train Acc:

Beam Width,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
Learning Rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Teacher Forcing Ratio,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train Accuracy,▁▂▃▄▅▅▆▆▆▇▇▇███
Train Loss,█▄▃▂▂▂▂▂▁▁▁▁▁▁▁
Validation Accuracy,▁▄▅▆▆▇▇▇▇▇█████
Validation Loss,█▄▂▂▂▂▁▁▁▁▁▁▁▁▁
Beam Width,5
Bidirectional,False
Epoch,15


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: tb23co9g with config:
wandb: 	batch_size: 64
wandb: 	beam_width: 3
wandb: 	bidirectional: False
wandb: 	cell_type: LSTM
wandb: 	dec_layers: 3
wandb: 	dropout: 0.3
wandb: 	embed_dim: 64
wandb: 	enc_layers: 2
wandb: 	epochs: 10
wandb: 	hidden_dim: 128
wandb: 	learning_rate: 0.001
wandb: 	optimizer: adam
wandb: 	teacher_forcing_ratio: 0.7


Epoch 1/10 | Train Loss: 0.0431, Train Acc: 0.13% | Val Loss: 0.0330, Val Acc: 0.85%
Epoch 2/10 | Train Loss: 0.0233, Train Acc: 6.47% | Val Loss: 0.0195, Val Acc: 13.82%
Epoch 3/10 | Train Loss: 0.0158, Train Acc: 17.00% | Val Loss: 0.0162, Val Acc: 22.62%
Epoch 4/10 | Train Loss: 0.0130, Train Acc: 23.32% | Val Loss: 0.0141, Val Acc: 27.44%
Epoch 5/10 | Train Loss: 0.0117, Train Acc: 27.99% | Val Loss: 0.0135, Val Acc: 29.33%
Epoch 6/10 | Train Loss: 0.0105, Train Acc: 31.31% | Val Loss: 0.0130, Val Acc: 32.65%
Epoch 7/10 | Train Loss: 0.0097, Train Acc: 34.13% | Val Loss: 0.0123, Val Acc: 32.20%
Epoch 8/10 | Train Loss: 0.0089, Train Acc: 36.88% | Val Loss: 0.0125, Val Acc: 33.13%
Epoch 9/10 | Train Loss: 0.0084, Train Acc: 39.07% | Val Loss: 0.0125, Val Acc: 34.52%
Epoch 10/10 | Train Loss: 0.0080, Train Acc: 41.03% | Val Loss: 0.0120, Val Acc: 35.66%


Beam Width,▁▁▁▁▁▁▁▁▁▁
Epoch,▁▂▃▃▄▅▆▆▇█
Learning Rate,▁▁▁▁▁▁▁▁▁▁
Teacher Forcing Ratio,▁▁▁▁▁▁▁▁▁▁
Train Accuracy,▁▂▄▅▆▆▇▇██
Train Loss,█▄▃▂▂▂▁▁▁▁
Validation Accuracy,▁▄▅▆▇▇▇▇██
Validation Loss,█▃▂▂▂▁▁▁▁▁
Beam Width,3
Bidirectional,False
Epoch,10


# Question 4 (10 Marks)

You will now apply your best model on the test data (You shouldn't have used test data so far. All the above experiments should have been done using train and val data only). 

(a) Use the best model from your sweep and report the accuracy on the test set (the output is correct only if it exactly matches the reference output). 


(b) Provide sample inputs from the test data and predictions made by your best model (more marks for presenting this grid creatively). Also upload all the predictions on the test set in a folder **predictions_vanilla** on your github project.

(c) Comment on the errors made by your model (simple insightful bullet points)

- The model makes more errors on consonants than vowels

- The model makes more errors on longer sequences

- I am thinking confusion matrix but may be it's just me!

- ...

In [4]:
import pandas as pd

def calculate_word_accuracy_from_ids(preds_ids, targets_ids, ignore_index=0):
    """
    Calculates word-level accuracy given token id tensors directly (both of shape [batch, seq_len]).
    Accuracy is computed as the percentage of sequences where all predicted tokens exactly match the target tokens.
    """
    # Create a mask where the target is not a padding token (ignore_index)
    mask = targets_ids != ignore_index

    # Apply the mask to both predictions and targets to ignore padded positions
    preds_masked = preds_ids * mask
    targets_masked = targets_ids * mask

    # Check if all tokens in the predicted sequence exactly match the target sequence
    sequence_correct = (preds_masked == targets_masked).all(dim=1)

    # Compute mean accuracy over the batch and convert to percentage
    word_accuracy = sequence_correct.float().mean().item() * 100

    return word_accuracy


def predict_and_log_test_examples_with_csv(model, test_path, src_vocab, tgt_vocab, device, num_examples=50, csv_save_path="predictions.csv"):
    # Set model to evaluation mode (no dropout, etc.)
    model.eval()

    # Unpack vocabularies: itos = index to string, stoi = string to index
    itos = tgt_vocab[1]
    stoi = src_vocab[0]

    # Load test dataset and randomly sample examples to evaluate
    test_data = read_dataset(test_path)
    examples = random.sample(test_data, num_examples)
    predictions_log = []

    # Lists to store predictions and targets for accuracy calculation
    preds_list = []
    trgs_list = []

    # ✅ List to collect prediction results for saving to CSV
    csv_data = []

    for src_text, tgt_text in examples:
        # Encode source sequence and add <eos> token, then convert to tensor
        src_tensor = torch.tensor(encode_sequence(src_text, stoi) + [stoi['<eos>']], device=device).unsqueeze(0)

        # Encode source sequence and adjust hidden state for decoder
        hidden = model.encoder(src_tensor)
        decoder_hidden = model.adjust_hidden_for_decoder(hidden)

        # Initialize decoder input with <sos> token
        input = torch.tensor([tgt_vocab[0]['<sos>']], device=device)

        decoded_tokens = []
        for _ in range(30):  # Max output length = 30
            output, decoder_hidden = model.decoder(input, decoder_hidden)
            top1 = output.argmax(1)  # Get index of highest scoring token
            if top1.item() == tgt_vocab[0]['<eos>']:
                break
            decoded_tokens.append(top1.item())
            input = top1  # Feed predicted token as next input

        # Convert predicted token indices to string
        prediction = decoded_tokens
        pred_str = ''.join([itos[idx] for idx in prediction])

        print(f"Input: {src_text} | Target: {tgt_text} | Prediction: {pred_str}")

        # Store current example's prediction for CSV saving
        csv_data.append({
            "Input": src_text,
            "Target": tgt_text,
            "Prediction": pred_str
        })

        # Create formatted HTML string for logging
        predictions_log.append(wandb.Html(f"<b>Input:</b> {src_text} &nbsp; <b>Target:</b> {tgt_text} &nbsp; <b>Pred:</b> {pred_str}"))

        # Encode the ground truth target with <eos>
        tgt_encoded = [tgt_vocab[0].get(ch, tgt_vocab[0]['<unk>']) for ch in tgt_text] + [tgt_vocab[0]['<eos>']]

        # Append predictions and targets as tensors
        preds_list.append(torch.tensor(prediction, device=device))
        trgs_list.append(torch.tensor(tgt_encoded, device=device))

    # Find max sequence length for padding
    max_len = max(max([p.size(0) for p in preds_list]), max([t.size(0) for t in trgs_list]))

    # Pad predictions and targets with zeros up to max_len
    preds_padded = pad_sequence([torch.cat([p, torch.full((max_len - p.size(0),), 0, device=device)]) if p.size(0) < max_len else p for p in preds_list], batch_first=True)
    trgs_padded = pad_sequence([torch.cat([t, torch.full((max_len - t.size(0),), 0, device=device)]) if t.size(0) < max_len else t for t in trgs_list], batch_first=True)

    # Calculate word-level accuracy across examples
    test_word_acc = calculate_word_accuracy_from_ids(preds_padded, trgs_padded)

    print(f"Test Word Accuracy on {num_examples} examples: {test_word_acc:.2f}%")

    # Log predictions and accuracy to wandb
    wandb.log({
        "Test Predictions": wandb.Html("<br>".join([str(p) for p in predictions_log])),
        "Test Word Accuracy": test_word_acc
    })

    # Save all predictions to a CSV file
    df = pd.DataFrame(csv_data)
    df.to_csv(csv_save_path, index=False)
    print(f" Saved predictions to {csv_save_path}")


def train_pred():
    # Initialize Weights & Biases run with config
    wandb.init(config={
        "embed_dim": 64,
        "hidden_dim": 128,
        "enc_layers": 3,
        "dec_layers": 3,
        "cell_type": "LSTM",
        "dropout": 0.2,
        "epochs": 15,
        "batch_size": 64,
        "bidirectional": False,
        "learning_rate": 0.001,
        "optimizer": "adam",
        "teacher_forcing_ratio": 0.7,
        "beam_width": 1
    })

    config = wandb.config
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Load train and validation datasets
    train_data = read_dataset("/kaggle/input/dakshina-dataset/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.train.tsv")
    dev_data = read_dataset("/kaggle/input/dakshina-dataset/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.dev.tsv")

    # Build vocabularies from source and target text
    src_vocab, tgt_vocab = build_vocab([src for src, _ in train_data]), build_vocab([tgt for _, tgt in train_data])

    # Instantiate Seq2Seq model
    model = Seq2Seq(len(src_vocab[0]), len(tgt_vocab[0]), config.embed_dim, config.hidden_dim,
                    config.enc_layers, config.dec_layers, config.cell_type, config.dropout, config.bidirectional).to(device)

    # Select optimizer
    if config.optimizer == "adam":
        optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
    elif config.optimizer == "nadam":
        optimizer = optim.NAdam(model.parameters(), lr=config.learning_rate)
    else:
        raise ValueError("Unsupported optimizer")

    # Define loss function, ignoring padding index
    criterion = nn.CrossEntropyLoss(ignore_index=0)

    # Training loop
    for epoch in range(config.epochs):
        model.train()
        total_loss = 0
        total_acc = 0

        # Shuffle training data at the beginning of each epoch
        random.shuffle(train_data)

        # Mini-batch training
        for i in range(0, len(train_data), config.batch_size):
            batch = train_data[i:i + config.batch_size]
            src, trg = prepare_batch(batch, src_vocab[0], tgt_vocab[0], device)

            optimizer.zero_grad()
            output = model(src, trg, teacher_forcing_ratio=config.teacher_forcing_ratio)

            # Ignore the <sos> token while calculating loss
            loss = criterion(output[:, 1:].reshape(-1, output.shape[-1]), trg[:, 1:].reshape(-1))
            acc = calculate_word_accuracy(output[:, 1:], trg[:, 1:])

            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            total_acc += acc

        # Calculate average metrics
        avg_train_loss = total_loss / len(train_data)
        avg_train_acc = total_acc / (len(train_data) // config.batch_size)

        # Evaluate on validation data
        val_loss, val_acc = evaluate(model, dev_data, src_vocab[0], tgt_vocab[0], device, criterion, config.batch_size)

        # Log metrics to wandb
        wandb.log({
            "Train Loss": avg_train_loss,
            "Train Accuracy": avg_train_acc,
            "Validation Loss": val_loss,
            "Validation Accuracy": val_acc,
            "Epoch": epoch + 1,
            "Learning Rate": config.learning_rate,
            "Teacher Forcing Ratio": config.teacher_forcing_ratio,
            "Optimizer": config.optimizer,
            "Bidirectional": config.bidirectional,
            "Beam Width": config.beam_width
        })

        print(f"Epoch {epoch + 1}/{config.epochs} | Train Loss: {avg_train_loss:.4f}, Train Acc: {avg_train_acc:.2f}% | Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

    # After training, evaluate on the test dataset
    test_path = "/kaggle/input/dakshina-dataset/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.test.tsv"
    predict_and_log_test_examples_with_csv(model, test_path, src_vocab, tgt_vocab, device, num_examples=100, csv_save_path="predictions_without_attention.csv")

    wandb.finish()


# ---------- Sweep Setup ----------
sweep_config = {
    'method': 'random',  # Random search over hyperparameters
    'metric': {'name': 'Validation Loss', 'goal': 'minimize'},
    'parameters': {
        'embed_dim': {'values': [64]},
        'hidden_dim': {'values': [128]},
        'enc_layers': {'values': [3]},
        'dec_layers': {'values': [3]},
        'cell_type': {'values': ['LSTM']},
        'dropout': {'values': [0.2]},
        'batch_size': {'value': 64},
        'epochs': {'value': 15},
        'bidirectional': {'values': [False]},
        'learning_rate': {'values': [0.001]},
        'optimizer': {'values': ['adam']},
        'teacher_forcing_ratio': {'values': [0.7]},
        'beam_width': {'values': [1]}
    }
}

# Initiate a sweep with wandb using the defined config
sweep_id = wandb.sweep(sweep_config, project="without_attention_best_model_test")

# Launch sweep agent (code is truncated here)
wandb.agent(sweep_id, function=train_pred, count=1)

Create sweep with ID: tawhomuk
Sweep URL: https://wandb.ai/viinod9-iitm/without_attention_best_model_test/sweeps/tawhomuk


wandb: Agent Starting Run: jzwnf922 with config:
wandb: 	batch_size: 64
wandb: 	beam_width: 1
wandb: 	bidirectional: False
wandb: 	cell_type: LSTM
wandb: 	dec_layers: 3
wandb: 	dropout: 0.2
wandb: 	embed_dim: 64
wandb: 	enc_layers: 3
wandb: 	epochs: 15
wandb: 	hidden_dim: 128
wandb: 	learning_rate: 0.001
wandb: 	optimizer: adam
wandb: 	teacher_forcing_ratio: 0.7


Epoch 1/15 | Train Loss: 0.0446, Train Acc: 0.01% | Val Loss: 0.0397, Val Acc: 0.09%
Epoch 2/15 | Train Loss: 0.0275, Train Acc: 3.74% | Val Loss: 0.0219, Val Acc: 12.16%
Epoch 3/15 | Train Loss: 0.0171, Train Acc: 16.02% | Val Loss: 0.0170, Val Acc: 23.85%
Epoch 4/15 | Train Loss: 0.0132, Train Acc: 23.96% | Val Loss: 0.0149, Val Acc: 28.24%
Epoch 5/15 | Train Loss: 0.0115, Train Acc: 29.00% | Val Loss: 0.0137, Val Acc: 29.99%
Epoch 6/15 | Train Loss: 0.0102, Train Acc: 32.85% | Val Loss: 0.0132, Val Acc: 32.86%
Epoch 7/15 | Train Loss: 0.0093, Train Acc: 35.82% | Val Loss: 0.0127, Val Acc: 34.86%
Epoch 8/15 | Train Loss: 0.0086, Train Acc: 39.05% | Val Loss: 0.0123, Val Acc: 36.47%
Epoch 9/15 | Train Loss: 0.0080, Train Acc: 41.71% | Val Loss: 0.0116, Val Acc: 36.47%
Epoch 10/15 | Train Loss: 0.0075, Train Acc: 43.68% | Val Loss: 0.0120, Val Acc: 37.27%
Epoch 11/15 | Train Loss: 0.0070, Train Acc: 45.81% | Val Loss: 0.0115, Val Acc: 37.27%
Epoch 12/15 | Train Loss: 0.0067, Train Acc:

Beam Width,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
Learning Rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Teacher Forcing Ratio,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test Word Accuracy,▁
Train Accuracy,▁▁▃▄▅▅▆▆▇▇▇▇███
Train Loss,█▅▃▂▂▂▂▂▁▁▁▁▁▁▁
Validation Accuracy,▁▃▅▆▆▇▇▇▇██████
Validation Loss,█▄▂▂▂▂▁▁▁▁▁▁▁▁▁
Beam Width,1
Bidirectional,False
